# Task 3

The solution proposed here is partly taken from Gurobi [Feature Selection case](https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/linear_regression/l0_regression.ipynb) to which you are referred for a more thorough machine learning pipeline. Here we focus only on the optimization problems.

## Solution Approach



### Sets and Indices

$i \in I=\{1,2,\dots,n\}$: Set of observations.

$j \in J=\{0,1,2,\dots,p\}$: Set of features, where the first ID corresponds to the intercept.

$\ell \in L = J \setminus \{0\}$: Set of features, where the intercept is excluded.


### Parameters

$s \in \mathbb{N}$: Number of features to include in the model, ignoring the intercept.


### Decision Variables

$\beta_j \in \mathbb{R}$: Weight of feature $j$, representing the change in the response variable per unit-change of feature $j$.

$z_\ell \in \{0,1\}$: 1 if weight of feature $\ell$ is exactly equal to zero, and 0 otherwise. Auxiliary variable used to manage the budget constraint.


### Training error

The training error is captured by a loss function, which can be defined as the Sum of Absolute Residuals (aka L1):

\begin{equation*}
 Z = \sum_{i \in I}\left|y_i-\sum_{j \in J}\beta_{j}x_{ij}\right|
\end{equation*}

or by the Residual Sum of Squares (RSS) (aka L2):

\begin{equation*}
 Z = \sum_{i \in I}\left(y_i-\sum_{j \in
J}\beta_{j}x_{ij}\right)^2 = \beta^T X^T X\beta- 2y^TX\beta+y^T y
\end{equation*}

Minimizing the loss function Z corresponds to solving an unconstrained optimization problem. The RSS formulation is the standard technique because it admits a closed form solution, which is given in the task statement and known as Ordinary Least Squares.
The Sum of Aboslute Errors is more robust with respect to outliers but it does not have a closed form solution mainly because the absolute error is not differentiable. However, it can be solved by Linear Programming.


### Feature selection

One way to set the $\beta$ terms to zero is to weight them in the objective function. This is what the Lasso approach does either using the L1 form:

\begin{equation*}
L=  \sum_{\ell=1}^p\left|\beta_\ell\right|
\end{equation*}

or the L2 form: 

\begin{equation*}
L= \sum_{\ell=1}^p\left(\beta_\ell\right)^2
\end{equation*}

This leads to solving another unconstrained optimization problem, namely:

\begin{equation*}
    \min Z + \lambda L
\end{equation*}

where $\lambda$ is a parameter that weights the importance of the Lasso term.
With the Lasso term also the RSS formalization of the training error becomes not solvable in closed form. However, the function is still convex and solved efficiently by (stochastic) gradient descent algorithms. This approach is used also in Deep Learning where the Lasso term is known as regularization term.


Alternatively, advances in mixed integer linear programming made it possible to model the feature selection more directly and precisely by means of constraints. 

Let us introduce binary variables $z_\ell$. They can be used to determine the features that are discarded from the regression: for each feature $\ell$, if $z_\ell=1$, then $\beta_\ell=0$. These are mutual exclusivity kind of constraints that Gurobi can handle in a specialized manner. In the solution provided by Gurobi this way of modeling the feature selection is called the $L_0$-norm and Gurobi provides modern [General Constraint Helper Functions](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/model.html#Model.addGenConstrNorm) to implement them. Here, however, we go the traditional way and linearize the mutal exclusivity constraints as follows:

\begin{equation*}
\left|\beta_\ell\right|\leq M (1-z_\ell)
\end{equation*}
Here the big-M is the upper bound of the $\beta$ variables, which are actually
unlimited. 

We can express each of these $|L|$ constraints as a [Special Ordered Set of type 1](https://docs.gurobi.com/projects/optimizer/en/current/concepts/modeling/constraints.html#subsubsectionsosconstraints), meaning that at most one variable is different from zero (variables can be both integer or continuous in SOS-1):

\begin{equation*}
(\beta_\ell, z_\ell): \text{SOS-1} \quad \forall \ell \in L
\tag{1}
\end{equation*}

This simply leaves to the solver the task of writing the M-constraints above.

Finally, in both cases we have the budget constraints satating that, exactly, $|L| - s$ feature coefficients must be equal to zero:

\begin{equation*}
\sum_{\ell \in L}z_\ell = |L| - s
\tag{2}
\end{equation*}

This model, by means of constraint 2, implicitly considers all ${{p} \choose s}$ feature subsets at once. However, we also need to find the value for $s$ that maximizes the performance of the regression on unseen observations. Notice that the training error decreases monotonically as more features are considered, so it is not advisable to use it as the performance metric. Instead, we should estimate the Mean Squared Error (MSE) via cross-validation. This metric is defined as $\text{MSE}=\frac{1}{n}\sum_{i=1}^{n}{(y_i-\hat{y}_i)^2}$, where $y_i$ and $\hat{y}_i$ are the observed and predicted values for the ith observation, respectively. Then, we will fine-tune $s$ using grid search, provided that the set of possible values is quite small.


### Linearization

All functions used above that have aboslute value terms can be linearized as follows. 

The objective function $\min Z =\sum_{i \in I}\left|y_i-\sum_{j \in J}\beta_{j}x_{ij}\right|$ becomes

\begin{align*}
\min &\sum_{i \in I} \epsilon_i^+ +\epsilon_i^-\\
 &y_i-\sum_{j \in J}\beta_{j}x_{ij} = \epsilon_i^+ -\epsilon_i^-\\
 &\epsilon_i^+, \epsilon_i^- \geq 0
\end{align*}

and the constraints $\left|\beta_\ell\right|\leq M (1-z_\ell)$ for all $\ell \in L$ become:

\begin{align*}
&\beta_\ell^+ +\beta_\ell^- \leq M(1-z_\ell)\\
 &\beta_{\ell} = \beta_\ell^+ -\beta_\ell^-\\
 &\beta_\ell^+, \beta_\ell^- \geq 0
\end{align*}


Hence, the overall regression problem with Z formulated with L1 and feature selection formulated as constraints becomes:

\begin{align*}
\min &\sum_{i \in I} (\epsilon_i^+ +\epsilon_i^-)\\
 &y_i-\sum_{j \in J}\beta_{j}x_{ij} = \epsilon_i^+ -\epsilon_i^- \quad \text{ for all } i \in I\\
&\beta_\ell^+ +\beta_\ell^- \leq M(1-z_\ell)\quad \text{ for all }\ell \in L\\
 &\beta_{\ell} = \beta_\ell^+ -\beta_\ell^- \quad \text{ for all } \ell \in L\\
 &\sum_{\ell \in L}z_\ell = |L| - s\\
 &\epsilon_i^+, \epsilon_i^- \geq 0 \quad \text{ for all }i \in I\\
 &\beta_\ell^+, \beta_\ell^- \geq 0 \quad \text{ for all } \ell \in L\\
 &\beta_j \in \mathbb{R} \quad \text{ for all } j \in J\\
 &z_\ell \in \{0,1\}\quad \quad \text{ for all } \ell \in L
\end{align*}
This is a mixed integer linear programming problem.
With Gurobi it is however also possible to solve the problem with a quadratic objecitve function as with the RSS formulation. Moreover, it is possible to specify the problem using matrices and vectors rather than scalars as shown here. See the solutions by Gurobi listed at the beginnning of this task.

# Python Implementation

Here we show the implementation of the mixed integer linear programming problem formulated above.

In [49]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math

import gurobipy as gp
from gurobipy import GRB

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing



In [50]:
# Load data and split into train (80%) and test (20%)
housing = fetch_california_housing()
X = housing['data']
y = housing['target']
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2,random_state=10101)

In [51]:
scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain_std = scaler.transform(Xtrain)
Xtest_std = scaler.transform(Xtest)

In [52]:
Xtrain_std.shape[1]

8

In [ ]:
# NOTE: This function assumes the design matrix features does not contain
#a column for the intercept
def milp(features, response, non_zero, sos=False, verbose=False):
    m = gp.Model("fitting")
    m.setParam("FeasibilityTol",1e-9)
    m.setParam("OptimalityTol",1e-9)
    m.setParam("IntFeasTol",1e-9)

    # Sets
    p = features.shape[1] # n of features
    n = features.shape[0] # n of observations (data points)

    I=range(n)
    J=range(p+1)
    L=range(1,p+1)

    # Create the decision variables
    beta = m.addVars(J,lb=-float('inf'), vtype=GRB.CONTINUOUS, name=f"beta_")
    zeta = m.addVars(L, vtype=GRB.BINARY, name="zeta_")

    # auxiliary to handle absolute values
    betap = m.addVars(L, lb=0.0, vtype=GRB.CONTINUOUS, name="betap_")
    betan = m.addVars(L, lb=0.0, vtype=GRB.CONTINUOUS, name="betan_")
            
    errp = m.addVars(I, lb=0.0, vtype=GRB.CONTINUOUS, name="errn_")
    errn = m.addVars(I, lb=0.0, vtype=GRB.CONTINUOUS, name="errp_")


    # The objective is to minimize 
    m.modelSense=gp.GRB.MINIMIZE
    m.setObjective(gp.quicksum(errp[i]+errn[i] for i in I))

    for i in I:
        m.addConstr(errp[i]-errn[i]==response[i]-beta[0]-gp.quicksum(beta[j]*features[i][j-1] for j in L ),name=f"loss_{i}")

    # Mutual exclusivity
    # Parameter
    if not sos:
        M=1e9 #float('inf')    
        for j in L:
            m.addConstr(betap[j]-betan[j]==beta[j],name=f"cbeta_{j}")
            m.addConstr(betap[j]+betan[j]<=M*(1-zeta[j]),name=f"M-constr_{j}")
    else:
        for j in L:
            # If zeta[i]=1, then beta[i] = 0
            m.addSOS(GRB.SOS_TYPE1, [zeta[j], beta[j]])
        
    # Budget constraints
    m.addConstr(gp.quicksum(zeta[j] for j in L)>=p-non_zero, name="budget")    

    # Solve
    m.write("feat_sel.lp")
    m.optimize()

    if m.status == gp.GRB.status.OPTIMAL:
        print('\nSum of Absolute Residuals (Training Error): %g' % m.ObjVal)
        print('\nbetas:')
        for j in J:
            #if math.fabs(beta[j].X)>0+0.0001:
            print('%s %g' % (j, beta[j].X))
        print('\nzetas:')
        for ell in L:
            #if zeta[j].X>0+0.0001:
            print('%s %g' % (ell, zeta[ell].X))
    else:
        print('No solution')


In [54]:
milp(Xtrain_std, ytrain, 5, True) # 5 features in the model

Set parameter FeasibilityTol to value 1e-09
Set parameter OptimalityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 24.6.0 24G419)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
FeasibilityTol  1e-09
IntFeasTol  1e-09
OptimalityTol  1e-09

Optimize a model with 16529 rows, 33057 columns and 181688 nonzeros (Min)
Model fingerprint: 0x7ef025cc
Model has 33024 linear objective coefficients
Variable types: 33049 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [9e-06, 1e+09]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Presolve time: 0.10s
Presolved: 16529 rows, 33057 columns, 181688 nonzeros
Variable types: 33049 continuous, 8 integer (8 b

The result is awkward. Rerunning using the SOS-2 constraints we obtain the
correct result.

-- 

The quadratic model:

In [55]:
# NOTE: This function assumes the design matrix features does not contain
#a column for the intercept
def miqp(features, response, non_zero, verbose=False):
    """
    Deploy and optimize the MIQP formulation of L0-Regression.
    """
    assert isinstance(non_zero, (int, np.integer))
    # Create a Gurobi environment and a model object
    with gp.Env() as env, gp.Model("", env=env) as regressor:
        samples, dim = features.shape
        assert samples == response.shape[0]
        assert non_zero <= dim

        # Append a column of ones to the feature matrix to account for the y-intercept
        X = np.concatenate([features, np.ones((samples, 1))], axis=1)

        # Decision variables
        norm_0 = regressor.addVar(lb=non_zero, ub=non_zero, name="norm")
        beta = regressor.addMVar((dim + 1,), lb=-GRB.INFINITY, name="beta") # Weights
        intercept = beta[dim] # Last decision variable captures the y-intercept

        regressor.setObjective(beta.T @ X.T @ X @ beta
                               - 2*response.T @ X @ beta
                               + np.dot(response, response), GRB.MINIMIZE)

        # Budget constraint based on the L0-norm
        regressor.addGenConstrNorm(norm_0, beta[:-1], which=0, name="budget")

        if not verbose:
            regressor.params.OutputFlag = 0
        regressor.params.timelimit = 60
        regressor.params.mipgap = 0.001
        regressor.optimize()

        coeff = np.array([beta[i].X for i in range(dim)])
        return intercept.X, coeff

In [ ]:
miqp(Xtrain_std, ytrain, 5, True) # 5 features in the model

Set parameter Username
Set parameter LicenseID to value 2762533
Academic license - for non-commercial use only - expires 2027-01-09
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 0.001
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 24.6.0 24G419)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  60
MIPGap  0.001

Optimize a model with 0 rows, 10 columns and 0 nonzeros (Min)
Model fingerprint: 0xa6f503be
Model has 9 linear objective coefficients and an objective constant of 9.3360800554079950e+04
Model has 45 quadratic objective terms
Model has 1 simple general constraint
  1 NORM
Variable types: 10 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [9e+02, 7e+04]
  QObjective range [1e-12, 6e+04]
  Bounds range     [5e+00, 5e+00]
  RHS range        [0e+00, 0e+00]
         Consider reformulating mod

(array(2.07596342),
 array([ 0.72353363,  0.12587989,  0.        ,  0.07955282,  0.        ,
         0.        , -0.99118496, -0.94202137]))

: 

That shows the intercept and the feature coefficients.

We refer to Gurobi solution for a comparison of the models' performance on the
housing learning task. 